In [ ]:
!pip3 install requests

In [ ]:
import requests 
import random
import string

base_url = "https://du-backend-t437.onrender.com/"

In [ ]:


def generate_ir_id():
    """Generate a random IR ID like IM8128."""
    prefix = "IM"
    suffix = "".join(random.choices(string.digits, k=4))
    return prefix + suffix

def create_ir(ir_id):
    """Send IR ID to the API."""
    path = "/api/add_ir_id"
    end_point = f"{base_url}{path}"
    payload = {"ir_id": ir_id}

    try:
        response = requests.post(end_point, json=payload, timeout=10)
        response.raise_for_status()
        print(f"Created {ir_id}: {response.json()}")
    except requests.exceptions.Timeout:
        print(f"{ir_id}: Request timed out.")
    except requests.exceptions.ConnectionError:
        print(f"{ir_id}: Failed to connect to the server.")
    except requests.exceptions.HTTPError as err:
        print(f"{ir_id}: HTTP error occurred: {err.response.text}")
    except Exception as e:
        print(f"{ir_id}: An error occurred: {e}")

def populate_ir_database(count=25):
    """Generate and send multiple IR IDs."""
    for _ in range(count):
        ir_id = generate_ir_id()
        create_ir(ir_id)


populate_ir_database(25)


In [ ]:
# Created IM7216: {'ir_id': 'IM7216'}
# Created IM6599: {'ir_id': 'IM6599'}
# Created IM1579: {'ir_id': 'IM1579'}
# Created IM9252: {'ir_id': 'IM9252'}
# Created IM2004: {'ir_id': 'IM2004'}
# Created IM4230: {'ir_id': 'IM4230'}
# Created IM7990: {'ir_id': 'IM7990'}
# Created IM0262: {'ir_id': 'IM0262'}
# Created IM1781: {'ir_id': 'IM1781'}
# Created IM1332: {'ir_id': 'IM1332'}
# Created IM9002: {'ir_id': 'IM9002'}
# Created IM4546: {'ir_id': 'IM4546'}
# Created IM4927: {'ir_id': 'IM4927'}
# Created IM6392: {'ir_id': 'IM6392'}
# Created IM2779: {'ir_id': 'IM2779'}
# Created IM9215: {'ir_id': 'IM9215'}
# Created IM8947: {'ir_id': 'IM8947'}
# Created IM8361: {'ir_id': 'IM8361'}
# Created IM2499: {'ir_id': 'IM2499'}
# Created IM0424: {'ir_id': 'IM0424'}
# Created IM6678: {'ir_id': 'IM6678'}
# Created IM9375: {'ir_id': 'IM9375'}
# Created IM8657: {'ir_id': 'IM8657'}
# Created IM5852: {'ir_id': 'IM5852'}
# Created IM1530: {'ir_id': 'IM1530'}

In [11]:
ir_ids = [
    "IM7216", "IM6599", "IM1579", "IM9252", "IM2004", "IM4230", "IM7990", "IM0262",
    "IM1781", "IM1332", "IM9002", "IM4546", "IM4927", "IM6392", "IM2779", "IM9215",
    "IM8947", "IM8361", "IM2499", "IM0424", "IM6678", "IM9375", "IM8657", "IM5852", "IM1530"
]

access_distribution = (
    [1] +                  # Admin
    [2] * 5 +               # LDC
    [3] * 8 +               # LS
    [4] * 4 +               # GC
    [5] * (len(ir_ids) - (1 + 5 + 8 + 4))  # IR
)

def register_ir(ir_id, access_level):
    path = "/api/register_new_ir"
    end_point = f"{base_url}{path}"
    data = {
        "ir_id": ir_id,
        "ir_name": f"Test User {ir_id}",
        "ir_email": f"{ir_id.lower()}@example.com",
        "ir_password": "TestPass123",
        "ir_access_level": access_level
    }
    
    try:
        response = requests.post(
            end_point,
            json=data,
            timeout=10
        )
        
        if response.ok:
            try:
                print(f"✅ Registered {ir_id} (Access: {access_level})")
            except ValueError:
                print(f"✅ Registered {ir_id} (Access: {access_level}) ")
        else:
            try:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code}")
            except ValueError:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code}")
    
    except requests.exceptions.RequestException as e:
        print(f"🚫 Failed {ir_id}: {e}")

def populate_test_irs():
    for ir_id, access_level in zip(ir_ids, access_distribution):
        register_ir(ir_id, access_level)

if __name__ == "__main__":
    populate_test_irs()

✅ Registered IM7216 (Access: 1)
✅ Registered IM6599 (Access: 2)
✅ Registered IM1579 (Access: 2)
✅ Registered IM9252 (Access: 2)
✅ Registered IM2004 (Access: 2)
✅ Registered IM4230 (Access: 2)
✅ Registered IM7990 (Access: 3)
✅ Registered IM0262 (Access: 3)
✅ Registered IM1781 (Access: 3)
✅ Registered IM1332 (Access: 3)
✅ Registered IM9002 (Access: 3)
✅ Registered IM4546 (Access: 3)
✅ Registered IM4927 (Access: 3)
✅ Registered IM6392 (Access: 3)
✅ Registered IM2779 (Access: 4)
✅ Registered IM9215 (Access: 4)
✅ Registered IM8947 (Access: 4)
✅ Registered IM8361 (Access: 4)
✅ Registered IM2499 (Access: 5)
✅ Registered IM0424 (Access: 5)
✅ Registered IM6678 (Access: 5)
✅ Registered IM9375 (Access: 5)
✅ Registered IM8657 (Access: 5)
✅ Registered IM5852 (Access: 5)
✅ Registered IM1530 (Access: 5)


In [ ]:
def getAllRegisteredIr():
    path = "/api/irs"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    
    if response.ok:
        try:
            print("Result:", response.json())
        except ValueError:
            print("Result (non-JSON):", response.text)
    else:
        try:
            print("Error:", response.json())
        except ValueError:
            print("Error (non-JSON):", response.text)

getAllRegisteredIr()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)